# kNN分类算法基础

1. 描述：如果一个样本在特征空间中的 k 个最相似即特征空间中最邻近)的样本中的大多数属于某一个类别，则该样本也属于这个类别
2. 步骤
    1. 分别计算此测试点与训练集中每一个样本的欧氏距离
    2. 按照距离的增序排序
    3. 选择前k个距离最小的点
    4. 确定这k个点所属标签出现的次数
    5. 以出现频率最高的类别作为当前点的预测分类
3. 参数k
    - k越小，模型越复杂，此时偏差小、方差大
    - k越大，模型越简单，此时偏差大、方差小（k最大为样本总数）
3. 复杂度
    - 时间复杂度
        1. 由于没有训练过程，所以训练时间复杂度为0
        2. 若训练集有m个样本，n个特征，则每预测一个新数据需要O(m*n)
        
4. 优点
    1. 精度高
    2. 没有对数据的分布假设
5. 缺点
    1. 计算复杂度高：效率低
    2. 高度数据相关：对异常值很敏感
    3. 预测结果不具有可解释性
    4. 维数灾难：随着维度增加，两个点的距离会越来越大
6. 特点
    - 非参数学习：没有训练过程，可认为训练集本身就是一个模型
    - 需要数据归一化
7. 优化
    - 采用更优的数据结构：KD-Tree、Ball-Tree（sklearn可设置）
    - 降维
8. 用途
    - 分类问题（多分类）
    - 回归问题

# scikit-learn中的kNN Classifier

### 核心步骤：
1. KNeighborsClassifier(n_neighbors=?)
2. kNN_classifier.fit(X, y)
3. kNN_classifier.predict(x_test)（需传入矩阵）

In [16]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier   #调用k近邻分类器

X_train = np.random.rand(10, 2)
y_train = np.array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])
x_test = np.random.rand(1, 2)
kNN_classifier = KNeighborsClassifier(n_neighbors=6) #n_neighbors即为k
kNN_classifier.fit(X_train, y_train)                 #拟合过程，返回此分类器对象

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=6, p=2,
           weights='uniform')

In [17]:
kNN_classifier.predict(x_test)                       #应传入矩阵，传入向量会报错

array([1])

# kNN中的超参数

- n_neighbors：k值，默认为5
- weights：权值方法，默认为'uniform'
    - uniform：平均权值
    - distance：以距离的倒数为权值
- p：闵可夫斯基距离（当weights='distance'时才生效）
    - p=1：曼哈顿距离（x轴距离加y轴距离的绝对值）
    - p=2：欧氏距离
- metric：度量方法，默认为'minkowski'（闵可夫斯基距离度量）

#### 确定k值和权值方法：

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

digits = datasets.load_digits()
X = digits.data
y = digits.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=666)

best_method = ""
best_score = 0.0
best_k = -1
for method in ["uniform", "distance"]:
    for k in range(1, 11):
        kNN_clf = KNeighborsClassifier(n_neighbors=k, weights=method)
        kNN_clf.fit(X_train, y_train)
        score = kNN_clf.score(X_test, y_test)
        if score > best_score:
            best_k = k
            best_score = score
            best_method = method
print("best_method = ", best_method)
print("best_k = ", best_k)
print("best_score = ", best_score)

best_method =  uniform
best_k =  4
best_score =  0.991666666667


#### 网格搜索（k*p）：寻找闵可夫斯基距离对应的p

In [7]:
%%time
best_p = -1
best_score = 0.0
best_k = -1

for k in range(1, 11):
    for p in range(1,7):
        kNN_clf = KNeighborsClassifier(n_neighbors=k, weights=method)
        kNN_clf.fit(X_train, y_train)
        score = kNN_clf.score(X_test, y_test)
        if score > best_score:
            best_k = k
            best_score = score
            best_p = p
print("best_p = ", best_p)
print("best_k = ", best_k)
print("best_score = ", best_score)

best_p =  1
best_k =  3
best_score =  0.988888888889
Wall time: 3.07 s


# scikit-learn中的kNN Regressor

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor

boston = datasets.load_boston()
X = boston.data
y = boston.target
X = X[y < 50.0]  #去除异常值
y = y[y < 50.0]  #同上

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=666)

kNN_reg = KNeighborsRegressor()  #k默认为5
kNN_reg.fit(X_train, y_train)
kNN_reg.score(X_test, y_test)  #根据默认参数所得的准确率

0.60267450508095299

In [5]:
param_grid = [ #total:10+50=60 candidates
    {
        'weights':['uniform'],
        'n_neighbors':[i for i in range(1, 11)]  #11-1=10 candidates
    },
    {
        'weights':['distance'],
        'n_neighbors':[i for i in range(1, 11)], #10*5=50 candidates
        'p':[i for i in range(1, 6)]
    }
]

kNN_reg = KNeighborsRegressor()
grid_search = GridSearchCV(kNN_reg, param_grid, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 60 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    2.0s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'weights': ['uniform'], 'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}, {'weights': ['distance'], 'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'p': [1, 2, 3, 4, 5]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [6]:
grid_search.best_params_

{'n_neighbors': 6, 'p': 1, 'weights': 'distance'}

In [14]:
grid_search.best_score_  #使用交叉验证的准确率

0.60603279917357411

In [15]:
grid_search.best_estimator_.score(X_test, y_test)  #R^2值

0.73542449060927706